In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE202665"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE202665"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE202665.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE202665.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE202665.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Time-resolved RNA signatures of CD4+ T cells in Parkinson's disease [mRNAarray]"
!Series_summary	"We employed time-resolved RNA expression analyses upon the activation of peripheral CD4+ T cells to track and functionally relate changes on cellular signaling in patients at different stages of PD"
!Series_overall_design	"A cohort of elderly male (age 53-85 yrs.; non-smokers), representative for different stages of Parkinson´s disease were tested (P1-5). Corresponding healthy controls (C1-5) (n=4) were matched for age (age 53-63 yrs.; non-smokers) and gender. One control was carried out twice (C1 and C1_2), representing independent replicates from the same healthy donor but from two different days of blood collection. CD4+ T cells were isolated from peripheral blood samples and were in vitro activated by αCD2/αCD3/αCD28 beads. Cellular samples were collected at 0, 2, 4, 8, 12 and 24 h after activation for subsequent RNA expression analyses."
Sample C

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title mentioning "mRNAarray" and description referring to "RNA expression analyses"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: Key 0 contains "disease state" information with Parkinson's disease vs Control
trait_row = 0

# For age: Key 3 contains age information
age_row = 3

# For gender: Key 2 contains gender information, but it shows all subjects are male (constant)
# Since this is a constant feature, we consider it not available for association studies
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert disease state to binary format (0: Control, 1: PD)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "parkinson" in value.lower() or "pd" in value.lower():
        return 1
    elif "healthy" in value.lower() or "control" in value.lower():
        return 0
    return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """
    Convert gender to binary format (0: Female, 1: Male)
    Not used in this dataset as gender is constant (all male)
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() in ["male", "m"]:
        return 1
    elif value.lower() in ["female", "f"]:
        return 0
    return None

# 3. Save Metadata
# Since trait_row is not None, trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM6128044': [1.0, 53.0], 'GSM6128045': [1.0, 53.0], 'GSM6128046': [1.0, 53.0], 'GSM6128047': [1.0, 53.0], 'GSM6128048': [1.0, 53.0], 'GSM6128049': [1.0, 53.0], 'GSM6128050': [1.0, 57.0], 'GSM6128051': [1.0, 57.0], 'GSM6128052': [1.0, 57.0], 'GSM6128053': [1.0, 57.0], 'GSM6128054': [1.0, 57.0], 'GSM6128055': [0.0, 63.0], 'GSM6128056': [0.0, 63.0], 'GSM6128057': [0.0, 63.0], 'GSM6128058': [0.0, 63.0], 'GSM6128059': [0.0, 63.0], 'GSM6128060': [0.0, 63.0], 'GSM6128061': [1.0, 75.0], 'GSM6128062': [1.0, 75.0], 'GSM6128063': [1.0, 75.0], 'GSM6128064': [1.0, 75.0], 'GSM6128065': [1.0, 75.0], 'GSM6128066': [1.0, 75.0], 'GSM6128067': [1.0, 85.0], 'GSM6128068': [1.0, 85.0], 'GSM6128069': [1.0, 85.0], 'GSM6128070': [1.0, 85.0], 'GSM6128071': [1.0, 85.0], 'GSM6128072': [1.0, 85.0], 'GSM6128073': [1.0, 76.0], 'GSM6128074': [1.0, 76.0], 'GSM6128075': [1.0, 76.0], 'GSM6128076': [1.0, 76.0], 'GSM6128077': [1.0, 76.0], 'GSM6128078': [1.0, 76.0], 'GSM6128079': [0.0, 63.0], 'GSM

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be numeric indices (1, 2, 3...) rather than actual gene symbols
# These are likely probe IDs or some other internal identifiers that would need to be mapped to gene symbols
# In human gene expression data, gene symbols would typically be named like 'BRCA1', 'TP53', etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1'], 'UNIGENE_ID': [nan, nan, nan, nan, 'Hs.189652'], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'unmapped', 'chr7:120901888-120901947'], 'CYTOBAND': [nan, nan, na

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the gene expression data and the gene annotation data, I need to identify the key columns for mapping
# Based on the previews, the gene expression data has ID values (1, 2, 3, ...) that align with the 'ID' column 
# in the gene annotation data, and the 'GENE_SYMBOL' column contains the gene symbols we need

# 2. Get a gene mapping dataframe
# Extract the ID column and the GENE_SYMBOL column from the gene annotation dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

print("Gene mapping dataframe preview:")
print(preview_df(mapping_df, n=10))

# 3. Convert probe-level measurements to gene expression data using the mapping
# This function handles many-to-many relations by splitting expression values when a probe maps to multiple genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("Number of genes after mapping:", len(gene_data))
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Gene mapping dataframe preview:
{'ID': ['5', '6', '7', '8', '12', '13', '14', '15', '16', '17'], 'Gene': ['CPED1', 'BCOR', 'CHAC2', 'IFI30', 'GPR146', 'SREK1IP1', 'lnc-ARMCX4-1', 'BAX', 'FPGT-TNNI3K', 'SYNGAP1']}


Number of genes after mapping: 29222
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols from the already mapped gene expression data from Step 6
# Apply normalization to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {normalized_gene_data.shape}")
print("First 5 normalized gene symbols:")
print(normalized_gene_data.index[:5])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already processed in Step 2
# We need to load the clinical data from the file that was saved in Step 2
if os.path.exists(out_clinical_data_file):
    clinical_data_processed = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data from file")
else:
    # If for some reason the file wasn't saved, recreate the clinical features using the same parameters as in Step 2
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Use the same conversion function and trait_row from Step 2
    def convert_trait(value):
        """Convert occupation to binary trait (exposure risk for Parkinson's)"""
        if not isinstance(value, str):
            return None
        
        value = value.lower().split(": ")[-1].strip()
        
        if "farmworker" in value:
            return 1  # Higher pesticide exposure (risk factor for Parkinson's)
        elif "manual worker" in value:
            return 0  # Lower pesticide exposure 
        else:
            return None
    
    # Use the exact same parameters as we determined in Step 2
    clinical_data_processed = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,  # From Step 2
        convert_trait=convert_trait,
        age_row=None,  # From Step 2
        convert_age=None,
        gender_row=None,  # From Step 2
        convert_gender=None
    )
    
    # Save it again just to be sure
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data_processed.to_csv(out_clinical_data_file)

print("Clinical data shape:", clinical_data_processed.shape)
print("Clinical data preview:")
print(preview_df(clinical_data_processed))

# 3. Link clinical and genetic data
# The clinical data is oriented with genes/traits as rows and samples as columns
# Transpose the normalized gene data to match this orientation (samples as columns)
genetic_data_t = normalized_gene_data

# Link clinical and genetic data vertically
linked_data = geo_link_clinical_genetic_data(clinical_data_processed, genetic_data_t)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")
print(f"Is the trait biased: {is_biased}")

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples comparing farmworkers (with higher pesticide exposure, a risk factor for Parkinson's) to manual workers."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable. Linked data was not saved.")

Gene expression data shape after normalization: (20778, 59)
First 5 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE202665.csv
Loaded clinical data from file
Clinical data shape: (2, 59)
Clinical data preview:
{'GSM6128044': [1.0, 53.0], 'GSM6128045': [1.0, 53.0], 'GSM6128046': [1.0, 53.0], 'GSM6128047': [1.0, 53.0], 'GSM6128048': [1.0, 53.0], 'GSM6128049': [1.0, 53.0], 'GSM6128050': [1.0, 57.0], 'GSM6128051': [1.0, 57.0], 'GSM6128052': [1.0, 57.0], 'GSM6128053': [1.0, 57.0], 'GSM6128054': [1.0, 57.0], 'GSM6128055': [0.0, 63.0], 'GSM6128056': [0.0, 63.0], 'GSM6128057': [0.0, 63.0], 'GSM6128058': [0.0, 63.0], 'GSM6128059': [0.0, 63.0], 'GSM6128060': [0.0, 63.0], 'GSM6128061': [1.0, 75.0], 'GSM6128062': [1.0, 75.0], 'GSM6128063': [1.0, 75.0], 'GSM6128064': [1.0, 75.0], 'GSM6128065': [1.0, 75.0], 'GSM6128066': [1.0, 75.0], 'GSM6128067': [1.0, 85.0], 'GSM6128068': [1.0, 85.0], 'GSM6128069': [1.0, 85.0], 'GSM6128070': [1.0, 85.0], 'GSM6128071': [1.0, 85.0], 'GSM6128072': [1.0, 85.0], 'GSM6128073': [1.0, 76.0], 'GSM61280

Data shape after handling missing values: (59, 20780)
For the feature 'Parkinsons_Disease', the least common label is '1.0' with 29 occurrences. This represents 49.15% of the dataset.
The distribution of the feature 'Parkinsons_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 66.0
  75%: 75.0
Min: 53.0
Max: 85.0
The distribution of the feature 'Age' in this dataset is fine.

Data shape after removing biased features: (59, 20780)
Is the trait biased: False


Linked data saved to ../../output/preprocess/Parkinsons_Disease/GSE202665.csv
